In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader

from torchvision import models, datasets, transforms


from tqdm import tqdm

In [8]:
device = torch.device("mps" if torch.backends.mps.is_available()
                      else "cuda" if torch.cuda.is_available() 
                      else "cpu")
print(device)

mps


In [10]:
# ResNetの事前学習済みモデルをロードする
resnet = models.resnet101(weights=models.ResNet101_Weights.DEFAULT).to(device)
optimizer = optim.Adam(resnet.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()

preprocess = models.ResNet101_Weights.DEFAULT.transforms()

categories = models.ResNet101_Weights.DEFAULT.meta["categories"]

In [11]:
resnet

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [5]:
resnet.fc.out_features = 10
resnet.fc

Linear(in_features=2048, out_features=10, bias=True)

In [6]:
train_transform = transforms.Compose([
    transforms.RandomCrop(size=32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.1),

    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2470, 0.2435, 0.2616))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465),
                         std=(0.2470, 0.2435, 0.2616))
])

In [ ]:
train = datasets.CIFAR10("../../data", train=True, transform=train_transform,
                         download=True)
test = datasets.CIFAR10("../../data", train=False, transform=test_transform,
                        download=True)

train_dataloader = DataLoader(train, batch_size=2048, shuffle=True,
                              generator=torch.Generator().manual_seed(42))

test_dataloader = DataLoader(test, batch_size=2048, shuffle=False,
                             generator=torch.Generator().manual_seed(42))

In [8]:
num_epochs = 10

resnet.train()
for epoch in range(1, num_epochs+1):
    running_loss = 0.0
    for images, labels in tqdm(train_dataloader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        pred = resnet(images)

        loss = criterion(pred, labels)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch} Loss: {running_loss}")

100%|██████████| 25/25 [01:13<00:00,  2.95s/it]


Epoch 1 Loss: 115.48596668243408


100%|██████████| 25/25 [14:16<00:00, 34.26s/it]  


Epoch 2 Loss: 27.932027399539948


100%|██████████| 25/25 [01:13<00:00,  2.94s/it]


...

Epoch 8 Loss: 10.139519214630127


100%|██████████| 25/25 [01:14<00:00,  2.98s/it]


Epoch 9 Loss: 9.1714488863945


100%|██████████| 25/25 [01:14<00:00,  2.97s/it]

Epoch 10 Loss: 8.428379118442535


In [ ]:
resnet.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for X, y in test_dataloader:
        X, y = X.to(device), y.to(device)
        logits = resnet(X)

        pred = torch.argmax(logits, dim=1)
        correct += (pred == y).sum().item()
        total += y.size(0)

    print(f"Acc: {correct / total}")

Acc: 0.8607


<img src="epoch5.png" style="width: 50%; height: auto;"/>